In [4]:

import pandas as pd
from qmt_data import QmtData
import os
import datetime
from xtquant import xtdata

In [7]:
today = datetime.datetime.today().strftime("%Y%m%d")

In [2]:
qmt_data = QmtData()
ticker_df = qmt_data.get_tickers()
stock_list = ticker_df["ticker"].tolist()



In [15]:
for ticker in stock_list:
    df = xtdata.get_divid_factors(ticker, start_time="20000101", end_time="20240820")
    if df.empty:
        continue
    df = (
        df
        .reset_index()
        .rename(columns={"index": "trade_time"})
        .drop(columns=["time"])
    )
    df["trade_time"] = pd.to_datetime(df["trade_time"])
    df.to_parquet(f"D:/qmt_datadir/adjust_factor/{ticker}.parquet", compression="snappy")

In [16]:
df

,trade_time,interest,stockBonus,stockGift,allotNum,allotPrice,gugai,dr
0,2021-06-29,0.4,0.0,0.0,0.0,0.0,0.0,1.003899
1,2022-06-29,0.5,0.0,0.0,0.0,0.0,0.0,1.004539
2,2023-06-29,1.0,0.0,0.0,0.0,0.0,0.0,1.009532
3,2024-07-01,1.5,0.0,0.0,0.0,0.0,0.0,1.014281
